In [1]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import nivapy3 as nivapy
import matplotlib.pyplot as plt
import critical_loads as cl

plt.style.use('ggplot')

/opt/conda/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  warnings.warn(


# Nitrogen deposition by land cover class

Generating summary N deposition statistics based on AR50 land cover. See the issue [here](https://github.com/JamesSample/critical_loads_2/issues/5) for details.

**Note:** In order to run this notebook for the entire country, it is necessary to log-in using a **high memory** machine.

## 1. Read raw datasets

In [2]:
# Connect to DSToolkit spatial data server
eng = nivapy.da.connect_postgis()

Connection successful.


In [3]:
# Connect to CL database
cl_eng = nivapy.da.connect_postgis(database='critical_loads')

Connection successful.


In [4]:
## Get Aust-Agder boundary. Just for testing - reduces computational time
#fyl = nivapy.da.read_postgis('cultural', 'norway_kartverket_fylker_2019_poly', eng)
#fyl = fyl.query('fylkes_navn == "Aust-Agder"')
#fyl

In [5]:
# Read AR50
ar50 = nivapy.da.read_postgis('physical', 'norway_nibio_ar50_poly', eng) # Add clip=fyl for sub-setting

# Remove invalid polys
ar50 = ar50[~ar50.geom.is_empty]

# Reproject to equal area proj for area calcs. Note that this takes a long time
ar50 = ar50.to_crs({'proj':'cea'})

ar50.head()

,ardyrking,arjordbr,arkartstd,arskogbon,artreslag,artype,arveget,fylke,komid,description,geom,id
0,99,98,AR50,98,39,50,52,12,1253,Snaumark: Fastmark med naturlig vegetasjonsdek...,"MULTIPOLYGON (((628192.280 5539299.548, 628203...",1
1,81,98,AR50,18,31,30,98,12,1253,Skog: Skogdekt areal,"MULTIPOLYGON (((613256.780 5536375.597, 613176...",2
2,81,98,AR50,18,32,60,98,12,1253,Myr: Areal som på overflata har preg av myr,"MULTIPOLYGON (((606540.248 5532623.341, 606468...",3
3,81,98,AR50,18,39,60,98,12,1253,Myr: Areal som på overflata har preg av myr,"MULTIPOLYGON (((605221.999 5532168.217, 605187...",4
4,81,98,AR50,18,31,60,98,12,1253,Myr: Areal som på overflata har preg av myr,"MULTIPOLYGON (((615747.044 5532413.275, 615749...",5


In [6]:
# Show series already in database
with pd.option_context("display.max_colwidth", -1):
    ser_grid = cl.view_dep_series(cl_eng)
    display(ser_grid)

,series_id,name,short_name,grid,description
0,1,Middel 1978-1982,7882,blr,Fordelt til BLR av NILU 2002
1,2,Middel 1992-1996,9296,blr,Fordelt til BLR av NILU 2002
2,3,Middel 1997-2001,9701,blr,Fordelt til BLR av NILU 2002
3,4,Middel 2002-2006,0206,blr,Fordelt til BLR av NILU 2008 (Wenche Aas)
4,5,Beregnet 2010,None,None,"Gøteborg protokollen 1999. ""Gamle"""
...,...,...,...,...,...
60,61,EMEP 1995 b,emep1995b,emep,"1995 data based on the EMEP 0.50x0.25 degree grid. 3 vegetation classes. Extracted by Max Posch, received Oct 2020"
61,62,EMEP 2000 b,emep2000b,emep,"2000 data based on the EMEP 0.50x0.25 degree grid. 3 vegetation classes. Extracted by Max Posch, received Oct 2020"
62,63,EMEP 2005 b,emep2005b,emep,"2005 data based on the EMEP 0.50x0.25 degree grid. 3 vegetation classes. Extracted by Max Posch, received Oct 2020"
63,64,EMEP 2010 b,emep2010b,emep,"2010 data based on the EMEP 0.50x0.25 degree grid. 3 vegetation classes. Extracted by Max Posch, received Oct 2020"


The series of interest here is **Middel 2012-2016 (new; hi-res)**, which has ID 28.

In [7]:
# Read N dep
n_gdf = cl.extract_deposition_as_gdf(28, 'nitrogen', cl_eng)
n_gdf = n_gdf[~n_gdf.geom.is_empty]  # Remove invalid polys

# Reproject to 'cea', as above
n_gdf = n_gdf.to_crs(ar50.crs)

n_gdf.head()

,geom,cell_id,ndep_mgpm2pyr,ndep_meqpm2pyr,ndep_kgphapyr
0,"MULTIPOLYGON (((734708.639 5395985.264, 734708...",58050655,840.0,59.957173,8.40
1,"MULTIPOLYGON (((743528.731 5393774.833, 742870...",58050665,911.0,65.024982,9.11
2,"MULTIPOLYGON (((756972.537 5395985.264, 756972...",58050675,1014.0,72.376874,10.14
3,"MULTIPOLYGON (((768104.486 5392967.593, 768104...",58050685,1070.0,76.374019,10.70
4,"MULTIPOLYGON (((779236.436 5390888.048, 779236...",58050695,1108.0,79.086367,11.08


## 2. Polygon intersections

In [8]:
%%time
gdf = gpd.overlay(ar50, n_gdf, how='intersection')
gdf['area_km2'] = gdf.geometry.area / 1E6
gdf['n_dep_poly_tonnespyr'] = gdf['ndep_mgpm2pyr'] * gdf['area_km2'] / 1000
gdf.head()

CPU times: user 1h 3min 55s, sys: 5.98 s, total: 1h 4min 1s
Wall time: 1h 4min 1s


,ardyrking,arjordbr,arkartstd,arskogbon,artreslag,artype,arveget,fylke,komid,description,id,cell_id,ndep_mgpm2pyr,ndep_meqpm2pyr,ndep_kgphapyr,geometry,area_km2,n_dep_poly_tonnespyr
0,99,98,AR50,98,39,50,52,12,1253,Snaumark: Fastmark med naturlig vegetasjonsdek...,1,60650565,875.0,62.455389,8.75,"POLYGON ((628192.280 5539299.548, 628203.806 5...",0.001311,0.001148
1,81,98,AR50,11,31,30,98,12,1253,Skog: Skogdekt areal,9,60650565,875.0,62.455389,8.75,"POLYGON ((627610.173 5542655.758, 627672.790 5...",0.004164,0.003643
2,81,98,AR50,18,99,30,98,12,1253,Skog: Skogdekt areal,10,60650565,875.0,62.455389,8.75,"POLYGON ((625207.207 5541447.567, 625293.489 5...",0.001794,0.001570
3,81,98,AR50,18,31,30,98,12,1253,Skog: Skogdekt areal,17,60650565,875.0,62.455389,8.75,"POLYGON ((626798.141 5542434.697, 626697.628 5...",0.003525,0.003085
4,81,98,AR50,18,33,30,98,12,1253,Skog: Skogdekt areal,21,60650565,875.0,62.455389,8.75,"POLYGON ((626936.190 5543780.357, 626945.904 5...",0.033821,0.029593


In [9]:
# AR50 themes of interest
themes = ['artype', 'artreslag', 'arveget', ['artype', 'artreslag']]

# Loop over themes
for theme in themes:
    # Group by AR50 class
    df = gdf.groupby(theme).agg({'n_dep_poly_tonnespyr':'sum',
                                 'area_km2':'sum'}).reset_index()
    
    # Print results
    print('############################################################################')
    print(f'Results for {theme}')
    print(df)
    print('')

    # Save
    if theme == ['artype', 'artreslag']:
        fpath = r'../ar50_summaries/artype_artreslag_ndep.csv'
    else:
        fpath = f'../ar50_summaries/{theme}.csv'
        
    df.to_csv(fpath, index=False)

############################################################################
Results for artype
  artype  n_dep_poly_tonnespyr       area_km2
0     10           1347.524063    1772.602125
1     20           7829.672193   12290.029021
2     30          57471.048182  124226.925938
3     50          39304.654125  136650.145148
4     60           7344.901462   21648.396497
5     70            798.317491    2830.077426
6     81           7406.516675   18391.762636
7     82           1672.832249    3344.193682
8     99              0.153353       0.610818

############################################################################
Results for artreslag
  artreslag  n_dep_poly_tonnespyr       area_km2
0        31          40235.902570   73129.904056
1        32          10997.236923   29855.882984
2        33           3553.017366    7289.486175
3        39          43850.037276  151808.396029
4        98          19055.016024   38629.275708
5        99           5484.409636   20441.798340



## 3. AR50 area summaries

**Added 26.01.2021.** See e-mails from Kari received 26.01.2021. We need to summarise the AR50 dataset by `ARTYPE` and `ARTRESLAG`.

Some of the code from above is repeated here for simplicity, which is slow.

In [11]:
df.sum()

artype                  102030303030506060606070818299
artreslag               989831323399393132333998989898
n_dep_poly_tonnespyr                     123175.619794
area_km2                                 321154.743292
dtype: object

In [ ]:
# Read AR50
ar50 = nivapy.da.read_postgis('physical', 'norway_nibio_ar50_poly', eng) # Add clip=fyl for sub-setting

# Remove invalid polys
ar50 = ar50[~ar50.geom.is_empty]

# Reproject to equal area proj for area calcs. Note that this takes a long time
ar50 = ar50.to_crs({'proj':'cea'})

# Calculate areas
ar50['area_km2'] = ar50.geometry.area / 1E6

ar50.head()

In [ ]:
# AR50 theme of interest
theme = ['artype', 'artreslag']

# Group by AR50 class
df = ar50.groupby(theme).agg({'area_km2':'sum'}).reset_index()

# Save
fpath = r'../ar50_summaries/artype_artreslag.csv'
df.to_csv(fpath, index=False)

df